# Python 101 @ SzISz VIII.
---
## Today: Dataframes, and visualization

### Act I: Get the data!

<img src="http://www.london24.com/polopoly_fs/1.3024317.1385128334!/image/4183113330.jpg_gen/derivatives/landscape_630/4183113330.jpg" align="left"></img>

Scrape the 2014 hungarian voting results!

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
VOTE_BASE = 'http://valasztas.hu/dyn/pv14/szavossz/hu/'
OVERALL = 'oevker.html'
BASE_URI = '../data/'

In [ ]:
vote_response = requests.get(VOTE_BASE + OVERALL)
print vote_response.status_code

In [ ]:
vote_soup = BeautifulSoup(vote_response.content, "lxml") 
containers = vote_soup.find('table', {'border': '1'}).findAll('tr')
containers[:5]

In [ ]:
# get the items out of the tablerows
rows = [row.findAll('td') for row in containers]
rows[:5]

In [ ]:
# we've got an unneeded first row, remove it!
rows = rows[1:]
rows[:5]

In [ ]:
# see the data in a table-like format
for row in rows[:5]:
    print [r.getText() for r in row]

In [ ]:
# we need the region, the subregion and the links
REGIONS = []
for row in rows:
    REGIONS.append([row[0].getText(), row[2].getText(), row[1].find('a').get('href')])
REGIONS[:5]

In [ ]:
print 'Number of regions:', len(REGIONS)

In [ ]:
results = []
# get the detailed informations
for region in REGIONS:
    region_response = requests.get(VOTE_BASE + region[2])
    region_soup = BeautifulSoup(region_response.content, "lxml")
    region_container = region_soup.find(text='A szavazatok száma jelöltenként').findNext('table').findAll('tr')
    region_rows = [row.findAll('td') for row in region_container]
    # every candidate will go to a new row
    for row in region_rows[1:]:
        results.append([region[0], region[1]] + [r.getText() for r in row][:-1]) # remove the last 'tick column'

In [ ]:
# let's look at the detailed information
print results[:5]
print '-' * 79
print 'Number of candidates:', len(results)

In [ ]:
cleaned_results =[]
# We have to transform the items
for row in results:
    cleaned_results.append(
        [item.replace(u'\xa0', u'').replace(u'%', u'').strip() # replace the unneeded characters
         for item in row]
    )

In [ ]:
cleaned_results[:5]

Now we can finally save it!

In [ ]:
import codecs
header = [u'region', u'subregion', u'subid', u'name', u'party', u'votes', u'votes %']
filename = 'vote2014.csv'
with codecs.open(BASE_URI + filename, 'w', 'utf-8') as csv:
    csv.write(u';'.join([u'"' + item + u'"' for item in header]) + u'\n')
    for row in cleaned_results:
        csv.write(u';'.join([u'"' + item + u'"' for item in row]) + u'\n')

# check if it is successfully created
import os
os.path.isfile(BASE_URI + filename)

---
### Act II: Use the pandas, Luke!

<img src="http://bconnelly.net/assets/img/pandas1.png" align="left">

In [ ]:
# import pandas! 
import pandas as pd

In [ ]:
#read in the data into a dataframe
data = pd.read_csv(BASE_URI + filename, quotechar='"', delimiter=';', encoding='utf-8')

In [ ]:
# show the first 5 rows
data.head()

In [ ]:
# we don't need the subid
del data['subid']

In [ ]:
# check the results
data[:5]

In [ ]:
# How many people voted?
data['votes'].sum()

In [ ]:
# let's get the sum of the votes for each party!
data.groupby('party').sum()

In [ ]:
# Cool! Order it by the number of votes!
party_votes = data[['party', 'votes']].groupby('party').sum().sort('votes', ascending=False)
party_votes

In [ ]:
len(data['party'].unique())

#### Let's draw!

In [ ]:
# use this `ipython magic` to draw the plots into the notebook
%matplotlib inline
# main plotting lib
import matplotlib.pyplot as plt
# make the plots nicer
try:
    import seaborn as sns
except:
    try:
        !pip install seaborn
    except:
        print 'Shit happens...'

In [ ]:
party_votes.plot(kind='bar')

In [ ]:
# Allright, let's get the ones with at least 10000 votes4
vote10k = party_votes[(party_votes['votes'] > 10000)]
vote10k.plot(kind='bar')

In [ ]:
# Let's get the top 6!
top6 = party_votes.head(6)
top6.plot(kind='bar')

---
### Act III: The devil lies in the details!

<img src="http://searchenginewatch.com/IMG/892/265892/panda-devil.png" width=300 height=300 align="left">

In [ ]:
# Let's use the regional data too!
regional = data[['party', 'region', 'votes']].groupby(['region', 'party']).sum()
regional

In [ ]:
# Let's only have the ones with more than 5000 votes!
regional5k = regional[regional['votes'] > 5000]
regional5k

In [ ]:
# We need to convert it into a plottable format
# pivot table
regional5k.stack()

In [ ]:
# then unpivot subtable
regional5k.stack().unstack('party')

In [ ]:
plt.rcParams['figure.figsize'] = 8, 6

In [ ]:
# that looks like something plottable!
regional5k.stack().unstack('party').plot(kind='barh')

---
### Final Act: The pandas is strong with this one! 

<img src="http://2.bp.blogspot.com/-pgK8KdMmSn8/TsFTOwrGk9I/AAAAAAAABAk/5ondVGyw6w8/s320/Darth+Panda.jpg" align="left">

In [ ]:
# Plot the number of voters in each region!
data[['region', 'votes']].groupby('region').sum().plot(kind='bar')

In [ ]:
# Who would win, if Fidesz doesn't participate in the election?
data_parties = (data[['party', 'votes']][data['party'] != "FIDESZ-KDNP"]
    .groupby('party')
    .sum()
    .sort(columns=['votes'], ascending=False))
data_parties[data_parties['votes'] == data_parties.values.max()]

In [ ]:
# Who would win by regions, if Fidesz doesn't participate in the election?
data_parties_2 = (data[['party', 'region', 'votes']][data['party'] != "FIDESZ-KDNP"]
    .groupby(['region', 'party'])
    .sum())

In [ ]:
data_parties_2